In [ ]:
# -*- coding: utf-8 -*-
from keras.preprocessing.image import img_to_array, load_img
from tensorflow.python.keras.models import load_model
from PIL import Image,ImageTk, ImageOps
from keras.models import load_model
from PIL import Image, ImageTk
from tkinter import messagebox
from tkinter import filedialog
import tkinter.filedialog
from pathlib import Path
import mediapipe as mp
import tkinter as tk
import numpy as np
import cv2
import os

        # アプリ情報（キャンバス情報、画面サイズ等）
class Application(tk.Frame):
    def __init__(self,master, video_source=0):
        super().__init__(master
)
        # ウィンドの初期サイズ指定
        self.master.geometry("380x170")
        # ウィンドの最小サイズ指定
        self.master.minsize(width = 380, height = 170)
        # ウィンドの最大サイズ指定
        self.master.maxsize(width = 380, height = 170)
        # ウィンドのタイトル指定
        self.master.title("指文字翻訳アプリ")
        
        # テキストボックスの Stringvar の用意
        self.trans_var = tk.StringVar()
        self.casca_var = tk.StringVar()
        self.model_var = tk.StringVar()
        self.camera_var = tk.StringVar()
        
        # ウィンドの位置設定
        self.columnconfigure(0, weight=1)
        self.columnconfigure(1, weight=1)
        self.columnconfigure(2, weight=1)

        # ウィンド生成
        self.create_widgets()
        # [mili seconds]
        self.delay = 15

        # 画面情報（ボタン表示、キャンバス表示）
    def create_widgets(self):
        
        # 各ボタンの横幅
        but_width = 10
        # 各要素ごとの間隔
        space = 5
        
        # 外枠の設定
        self.frame_btn = tk.LabelFrame(self.master)
        # 位置
        self.frame_btn.place(x=10, y=10)
        # 縦横の幅
        self.frame_btn.configure(width = 360, height = 150)
        # 親位置
        self.frame_btn.grid_propagate(0)

        # 閉じるボタン
        self.can_clo = tk.Button(self.frame_btn, text = 'モデル選択')
        # ボタンの設定
        self.can_clo.configure(width = but_width,
                               height = 1,
                               command = self.select_model)
        # ボタンの位置
        self.can_clo.grid(column = 2,
                          row = 2,
                          padx = 20,
                          pady = space)
        
        # 画像ボタン
        self.can_clo = tk.Button(self.frame_btn, text = '分類器選択')
        # ボタンの設定
        self.can_clo.configure(width = but_width,
                               height = 1,
                               command = self.select_cascade)
        # ボタンの位置
        self.can_clo.grid(column = 2,
                          row = 1,
                          pady = space)

        # 動画ボタン
        self.start = tk.Button(self.frame_btn, text = '起動')
        # ボタンの設定
        self.start.configure(width = but_width,
                             height = 1,
                             command = self.botton)
        # ボタンの位置
        self.start.grid(column = 2,
                        row = 0,
                        pady = space)
        
        # テキスト消去
        del_button = tk.Button(self.frame_btn, text = '消しゴム')
        # テキストの設定
        del_button.configure(width = but_width,
                             height = 1,
                             command = self.btn_clear_click)
        # テキストの位置
        del_button.grid(column = 2,
                        pady = space)
        # パディング
        padding = {'padx': 5, 'pady': 5}
        
        # 「翻訳」の場所指定
        tk.Label(self.frame_btn, text = '翻訳').grid(column = 0, row = 0, **padding)
        tk.Label(self.frame_btn, text = '分類器').grid(column = 0, row = 1, **padding)
        tk.Label(self.frame_btn, text = 'モデル').grid(column = 0, row = 2, **padding)
        tk.Label(self.frame_btn, text = 'カメラ').grid(column = 0, row = 3, **padding)

# ──────── テキストボックスの設定 ────────#
        # 翻訳
        self.trans_entry = tk.Entry(self.frame_btn, textvariable = self.trans_var)
        # テキストボックスを少し大きくしフォントを変更
        self.trans_entry.configure(font = ("Courier", 11))
        # 位置
        self.trans_entry.grid(column = 1, row = 0, **padding)
        # フォーカス切り替え
        self.trans_entry.focus()
        
        # 以下同文
        
        # 分類器
        self.casca_entry = tk.Entry(self.frame_btn, textvariable = self.casca_var)
        self.casca_entry.grid(column = 1, row = 1, **padding)
        self.casca_entry.focus()
        
        # モデル
        self.model_entry = tk.Entry(self.frame_btn, textvariable = self.model_var)
        self.model_entry.grid(column = 1, row = 2, **padding)
        self.model_entry.focus()
        
        # カメラ
        self.camera_entry = tk.Entry(self.frame_btn, textvariable = self.camera_var)
        self.camera_entry.grid(column = 1, row = 3, **padding)
        self.camera_entry.focus()
# ───────────────────────────#
    # 翻訳テキストボックスを空白にする
    def btn_clear_click(self):
        # 翻訳のテキストボックスに空文字を上書きする
        self.trans_var.set("")
        

    # 分類器選択
    def select_cascade(self, event = None):
        # ウィンドから指定したファイルのパスを取得する
        filename = filedialog.askopenfilename(title = "カスケード分類器を選択",
            filetypes = [("", "*.xml")], # ファイルフィルタ（.xmlファイルのみ）
            initialdir = os.path.abspath(Path().resolve()) # 自分自身のディレクトリ
        )
        # 分類器のテキストボックスにパスを上書きする
        self.casca_var.set(filename)
        
    # モデル選択
    def select_model(self, event = None):
        # # ウィンドから指定したファイルのパスを取得する
        filename = filedialog.askopenfilename(title = "モデルファイルを選択",
            filetypes = [("", "*.json .pickle .pkl .hdf5 .h5")], # ファイルフィルタ
            initialdir = os.path.abspath(Path().resolve()) # 自分自身のディレクトリ
        )
        # モデルのテキストボックスにパスを上書きする
        self.model_var.set(filename)
        
# ──────────── チェック＆エラーメッセージ表示 ───────────────#

    # 分類器パスが空白かどうかの確認
    def cascade_check(self, CasPath, camera):
        # 空白の場合
        if CasPath == "":
            # エラーメッセージ表示
            messagebox.showerror('エラー', 'カスケードファイルが選択されていません')
            camera.release()
        # テキストボックスに値が入っているが、書き方が正しくない場合
        elif os.path.exists(CasPath) == False:
            # エラーメッセージ表示
            messagebox.showerror('エラー', 'カスケードファイルが見つかりません')
            camera.release()
        else:
            pass
            
    # モデルパスが空白かどうかの確認
    def model_check(self, ModPath, camera):
        if ModPath == "":
            messagebox.showerror('エラー', 'モデルが選択されていません')
            camera.release()
        elif os.path.exists(ModPath) == False:
            messagebox.showerror('エラー', 'モデルが見つかりません')
            camera.release()
        else:
            pass
    
    # カメラが指定されているかどうかの確認
    def camera_check(self, CamNum):
        # 
        if CamNum == "":
            # 
            messagebox.showerror('エラー', 'カメラが指定されていません')
        # 
        elif CamNum.isdecimal() == False:
            # 
            messagebox.showerror('エラー', 'カメラが見つかりません')
        else:
            # 
            camera = cv2.VideoCapture(int(CamNum))
        # 
        return camera
        
# ─────────────────────────────────────────#
    # メイン処理
    def botton(self):
        
        # 変数の準備
        array = []
        hoge = ""
        fuga = ""
        
        # テキストボックスの値を取得
        cas_get = self.casca_entry.get()
        mod_get = self.model_entry.get()
        cam_get = self.camera_entry.get()
        
        # チェック処理して値を代入
        camera = self.camera_check(cam_get)
        
        # チェック処理
        self.cascade_check(cas_get, camera)
        self.model_check(mod_get, camera)
        
        # 手を認識するときの設定
        hands = mp.solutions.hands.Hands(
            static_image_mode = False,
            max_num_hands = 2,
            min_detection_confidence = 0.9,
            min_tracking_confidence = 0.9)
        
        # カスケードファイルの読み込み
        cascade = cv2.CascadeClassifier(cas_get)
        # モデルの読み込み
        model = load_model(mod_get)
        
        while camera.isOpened():

            # カメラ映像取得
            ret, frame = camera.read()

            # 映像を左右反転されカラーでふれー
            frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
            # 手の認識で設定した変数を使って手を取る
            results = hands.process(frame)
            # 
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            # 手にポイントを付ける
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:

                    mp.solutions.drawing_utils.draw_landmarks(
                        frame,
                        hand_landmarks,
                        mp.solutions.hands.HAND_CONNECTIONS,
                        mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                        mp.solutions.drawing_styles.get_default_hand_connections_style()
                    )

            # 映像を白黒で読み込む
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            #「cascade」にはカスケードファイル読み込んだものが入っている（パスの成否問わず）
            #「face」 = カスケードファイルを使って顔の検出をする処理
            # 引数は「どの画像から検出するか」「スケールの縮小量」「顔検出のしきい値」「この値より小さい物体（顔）は無視」
            face = cascade.detectMultiScale(gray, scaleFactor = 1.1, minNeighbors = 3, minSize = (90, 90))
            # 検出処理「face」を動かし続ける
            for (x,y,w,h) in face:
                # 認識した場所を赤枠で囲う
                cv2.rectangle(frame,(x, y),(x + 130, y + 130),(0,0,200),3)
                # 認識した場所を「128⋆128」で切り取る
                frame_cut = frame[y + 5:y + 133,x + 5:x + 133]
                
                # 画像を配列に変換
                img = img_to_array(frame_cut)
                # 正規化
                img_nad = img_to_array(img)/255
                img_nad = img_nad[None, ...]

                # ラベルの準備
                label=['a','thi','thu']
                
                # 一度に認識する量が多いとこの行がエラーを出していたため
                # トライエクセプトで囲いエラーが出たらもう一度最初から
                try:
                    # 画像の文字が何なのかを予測
                    pred = model.predict(img_nad, batch_size=1, verbose=0)
                except:
                    continue
                
                # 予測されたラベルを取得
                pred_label = label[np.argmax(pred[0])]
                
                # 変数とラベルが同じでなけらば
                if hoge != pred_label:
                    
                    # hoge 変数に値を代入
                    hoge = pred_label
                    # 配列をクリア
                    array.clear()
                    # 空っぽの配列にラベルを入れる
                    array.append(pred_label)
                    # 認識したもののラベルを表示
                    cv2.putText(frame,pred_label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255), thickness=2)
                # 変数をラベルが同じであれば
                else:
                    # 配列にラベルを入れる
                    array.append(pred_label)
                    # 認識したもののラベルを表示
                    cv2.putText(frame,pred_label,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1,(255, 255, 255), thickness=2)

                    # 配列の長さが15以上になったら
                    if len(array) >= 15: 
                        # 翻訳テキストボックスに値を追加
                        self.trans_entry.insert(tkinter.END, pred_label)
                        # fuga 変数にラベルを入れる
                        fuga += pred_label
                        # 配列をクリア
                        array.clear()
            
            # 翻訳テキストボックスに入る文字列を画面端に常時表示
            cv2.putText(frame, fuga, (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), thickness=2)
            #「Frame」ウィンドウにキャプチャし続けている画像表示
            cv2.imshow('camera', frame) 

            # cv2.waitKeyは()で指定した値（ms）の間キー入力を待つ
            # 今回は1msごとで「q」キー入力を待つ
            #「q」が押されたら
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # カメラ停止
        camera.release()
        # カメラ映像を映しているウィンドウを閉じる
        cv2.destroyAllWindows()     

# GUIのスタート
def main():
    # メインウィンドウ（名前：tk）の設定
    root = tk.Tk()
    # アプリを動かす
    app = Application(master = root)#Inherit
    # プログラムを無限ループさせており、アプリを意図的に終了させない限りアプリを待機させる役割
    app.mainloop()

if __name__ == "__main__":
    main()